<a href="https://colab.research.google.com/github/Chan0226/Recommendation-system/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/rounakbanik/the-movies-dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movie =  pd.read_csv('/content/ratings.csv',low_memory=False)
movie = movie.rename(columns={'movieId' : 'id'})
print(movie.shape)
movie.head()

(4324793, 4)


,userId,id,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
movie = movie.sort_values(by = 'timestamp').reset_index()

movie.head()

,index,userId,id,rating,timestamp
0,4324792,44815,4995,4.5,108544
1,3662481,38150,1176,4.0,789652004
2,4313954,44717,1079,3.0,789652009
3,4313911,44717,47,5.0,789652009
4,4313905,44717,21,3.0,789652009


In [ ]:
movie_meta = pd.read_csv('/content/movies_metadata.csv', low_memory=False)
movie_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movie_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
movie_meta['id'] = movie_meta['id'].astype(str)
movie['id'] = movie['id'].astype(str)

In [ ]:
movie_meta = pd.merge(movie, movie_meta[['id', 'original_title']], how = 'left', on = 'id')

In [ ]:
movie_meta.head()

,index,userId,id,rating,timestamp,original_title
0,4324792,44815,4995,4.5,108544,Boogie Nights
1,3662481,38150,1176,4.0,789652004,NaN
2,4313954,44717,1079,3.0,789652009,NaN
3,4313911,44717,47,5.0,789652009,NaN
4,4313905,44717,21,3.0,789652009,The Endless Summer


In [ ]:
movie_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4324973 entries, 0 to 4324972
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   index           int64  
 1   userId          int64  
 2   id              object 
 3   rating          float64
 4   timestamp       int64  
 5   original_title  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 231.0+ MB


In [ ]:
movie_meta = movie_meta[movie_meta['original_title'].notnull()].reset_index()

In [ ]:
# movie_meta.groupby(['userId'])['rating'].agg({'sum','count'})
agg = movie_meta.groupby(['userId'])['original_title'].agg({'unique'})
agg = pd.DataFrame(agg)
agg.head()

,unique
userId,
1,"[Young and Innocent, Shuang ma lian huan, Cesa..."
2,"[La passion de Jeanne d'Arc, La belle et la bê..."
3,"[I Love You to Death, Once Were Warriors, Mons..."
4,"[Muxmäuschenstill, Batman & Robin, Hidalgo, 12..."
5,"[Star Trek III: The Search for Spock, The Curs..."


In [ ]:
agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20082 entries, 1 to 20452
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   original_title  20082 non-null  object
dtypes: object(1)
memory usage: 313.8+ KB


In [ ]:
# Word2vec -- 문자
# agg.index = agg.index.astype(str)
test = []
for i in agg['unique'].values:
  test.append(list(map(str,i)))

test[1:10]  


[["La passion de Jeanne d'Arc",
  'La belle et la bête',
  'Donnie Darko',
  'Jarhead',
  'The 39 Steps',
  'Four Rooms',
  'Almost Famous',
  'Mrs. Doubtfire',
  'Monty Python and the Holy Grail',
  'Interview with the Vampire',
  '英雄',
  "Pirates of the Caribbean: Dead Man's Chest",
  'Hable con ella',
  'The Matrix Revolutions',
  'Parenthood',
  'A Nightmare on Elm Street',
  'Night on Earth'],
 ['I Love You to Death',
  'Once Were Warriors',
  'Monsoon Wedding',
  'Reservoir Dogs',
  'Sleepless in Seattle',
  'The Third Man',
  'My Super Ex-Girlfriend',
  'Fools Rush In'],
 ['Muxmäuschenstill',
  'Batman & Robin',
  'Hidalgo',
  '12 + 1',
  'Tuya de hun shi',
  'Reign Over Me',
  'Rebecca',
  'The 13th Warrior',
  'Der Tunnel',
  'Furankenshutain Tai Chitei Kaijū Baragon',
  'ฟ้าทะลายโจร',
  'Der rote Elvis',
  'Young and Innocent',
  'The Searchers',
  'Shriek If You Know What I Did Last Friday the Thirteenth',
  "L'important c'est d'aimer",
  'The Prisoner of Zenda',
  'Don Juan

In [ ]:
from gensim.models import Word2Vec
model =  Word2Vec(test,size = 20, window = 5, min_count =1, workers=4,iter=10,sg=1)

In [ ]:
model.wv.most_similar(positive=['Taxi 3'], topn=10)

[('Rent', 0.9525529146194458),
 ('Stuck on You', 0.9471043348312378),
 ('Erkan & Stefan in Der Tod kommt krass', 0.9425098896026611),
 ('Escape from the Planet of the Apes', 0.906892716884613),
 ("L'Enfant sauvage", 0.8924970030784607),
 ("La Marche de l'empereur", 0.8875250220298767),
 ('綠草地', 0.8772715926170349),
 ('Transformers', 0.8616079092025757),
 ('Bordertown', 0.8615713119506836),
 ('Secret Window', 0.8482496738433838)]